# Markov model
Simon Frost (@sdwfrost), 2020-04-27

## Introduction

The Markov model approach taken here is:

- Stochastic
- Discrete in time
- Discrete in state

## Libraries

In [ ]:
using DifferentialEquations
using SimpleDiffEq
using Distributions
using Random
using DataFrames
using StatsPlots
using BenchmarkTools

## Utility functions

In [ ]:
@inline function rate_to_proportion(r::Float64,t::Float64)
    1-exp(-r*t)
end;

## Transitions

In [ ]:
function sir_markov!(du,u,p,t)
    (S,I,R) = u
    (β,c,γ,δt) = p
    N = S+I+R
    ifrac = rate_to_proportion(β*c*I/N,δt)
    rfrac = rate_to_proportion(γ,δt)
    infection=rand(Binomial(S,ifrac))
    recovery=rand(Binomial(I,rfrac))
    @inbounds begin
        du[1] = S-infection
        du[2] = I+infection-recovery
        du[3] = R+recovery
    end
    nothing
end;

## Time domain

Note that even though we're using fixed time steps, `DifferentialEquations.jl` complains if I pass integer timespans, so I set the timespan to be `Float64`.

In [ ]:
δt = 0.1
nsteps = 400
tmax = nsteps*δt
tspan = (0.0,nsteps)
t = 0.0:δt:tmax;

## Initial conditions

In [ ]:
u0 = [990,10,0]; # S,I,R

## Parameter values

In [ ]:
p = [0.05,10.0,0.25,δt]; # β,c,γ,δt

## Random number seed

In [ ]:
Random.seed!(1234);

## Running the model

In [ ]:
prob_markov = DiscreteProblem(sir_markov!,u0,tspan,p)

In [ ]:
sol_markov = solve(prob_markov,solver=FunctionMap());

## Post-processing

We can convert the output to a dataframe for convenience.

In [ ]:
df_markov = DataFrame(sol_markov')
df_markov[!,:t] = t;

## Plotting

We can now plot the results.

In [ ]:
@df df_markov plot(:t,
    [:x1 :x2 :x3],
    label=["S" "I" "R"],
    xlabel="Time",
    ylabel="Number")

## Benchmarking

In [ ]:
@benchmark solve(prob_markov,solver=FunctionMap)